<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 6. Dashboards</h2></div>

So far in this tutorial, we have seen how to generate plots with `.plot` or `.hvplot`, how to compose these plots together into layouts and overlays, how to apply linked selections and how to control visualizations with Panel widgets using `.interactive`. In this notebook, we will learn how to put all these pieces together to display (and serve) these components in a dashboard.

## Panel `pane` objects

So far we have only seen Panel used as a source of widgets but Panel also offers `pane` objects that can display various types of data. First let's import panel and load the extension:

In [ ]:
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr
import holoviews as hv

pn.extension('tabulator', template='material') # TODO: EXPLAIN
pn.state.template.sidebar_width = 250 # TODO: EXPLAIN
pn.config.sizing_mode = 'stretch_width' # TODO: EXPLAIN

import colorcet as cc
import hvplot.xarray  # noqa
import hvplot.pandas # noqa
pn.extension()

One example of a `pane` is a `Markdown` pane which displays Markdown:

In [ ]:
pn.pane.Markdown('## Earthquake Dashboard')

Another type of `pane` is the `PNG` pane which can display PNG images:

In [ ]:
pn.pane.PNG('../assets/usgs_logo.png', height=130)

## Using `pn.panel`

Instead of having to select the pane type explicitly, you can use the `pn.panel` function which will try to guess the appropriate representation given the input. For instance, here we generate the same two panels using `pn.panel` and grab handles on the resulting objects:

In [ ]:
dashboard_title = pn.panel('## Earthquakes')
usgs_logo = pn.panel('../assets/usgs_logo.png', height=130)

#### Exercise

Confirm that these two objects are of type `Markdown` and `PNG` respectively by using the `type` built-in. Explore using different markdown syntax such as *italic*, **bold** or adding bullet points. Finally, try displaying your own PNG image with a `PNG` pane.

## `Panel` objects

In addition to `pane` objects, Panel offers containers of type `panel` which allows you to position your components into various layouts. For instance, we can put a small version of our title and logo into a `Row` layout:

In [ ]:
header = pn.Row(dashboard_title, pn.pane.PNG('../assets/usgs_logo.png', height=40))
header

Next let us load the earthquake dataset and make a basic plot of the sort we have seen earlier on in the tutorial:

In [ ]:
df = pd.read_parquet('../data/earthquakes.parq')
df = df.set_index('time').tz_convert(None).reset_index()
small_df = df.sample(frac=.01)

ds = xr.open_dataarray('../data/raster/gpw_v4_population_density_rev11_2010_2pt5_min.nc')
cleaned_ds = ds.where(ds.values != ds.nodatavals).sel(band=1)
cleaned_ds.name = 'population'

In [ ]:
sample_points = small_df.hvplot.points(x='longitude', y='latitude', c='mag', cmap=cc.CET_L4)
rasterized_pop = cleaned_ds.hvplot.image(rasterize=True, logz=True, clim=(1, np.nan)).opts(bgcolor='black')
earthquake_example = rasterized_pop * sample_points
earthquake_example

Now we can combine this plot with our header in a `pn.Column`:

In [ ]:
mini_dashboard = pn.Column(header, earthquake_example)
mini_dashboard

## Showing and serving dashboards

Now`mini_dashboard` is a panel object that can be displayed or served as a simple dashboard. To view the dashboard in a new tab, you can simply call `.show()`:

In [ ]:
# mini_dashboard.show()

If instead of `.show()` you use the `.servable()` method, you can serve the dashboard from this notebook using the command:

```bash
$ panel serve 07_Dashboard.ipynb
```

We will use this to serve a more sophisticated dashboard built in this notebook.

## A visual earthquake filter

From the last notebook, first subset the data:

In [ ]:
from datashader.utils import lnglat_to_meters
subset_df = df[
            (df['mag']   > 4) &
            (df['time'] >= pd.Timestamp('2017-01-01')) &
            (df['time'] <= pd.Timestamp('2018-01-01'))
]

x, y = lnglat_to_meters(subset_df['longitude'], subset_df['latitude'])
subset_projected = subset_df.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})])

Declare the panel widgets:

In [ ]:
date_subrange = pn.widgets.DateRangeSlider(name='Date', 
                                        start=subset_df.time.iloc[0], 
                                        end=subset_df.time.iloc[-1])
mag_subrange = pn.widgets.FloatSlider(name='Magnitude', start=3, end=9, value=3)

Create an interactive `DataFrame` and use `hvplot` to generate a visualization of earthquakes plotted on a map and controlled with widgets:

In [ ]:
subset_dfi = subset_projected.interactive(sizing_mode='stretch_width')
filtered_subrange = subset_dfi[
    (subset_dfi['mag']   > mag_subrange) &
    (subset_dfi['time'] >= date_subrange.param.value_start) &
    (subset_dfi['time'] <= date_subrange.param.value_end)
]

geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500, tiles='ESRI'
)

This `geo` object works in panel layouts which means we can now add a header to it:

In [ ]:
pn.Column(header, geo)

We now have a functional dashboard that we can show with `.show()` or serve with `.servable()`.

## Final Dashboard

We can now put all the concepts we have learned together to make a sophisticated, interactive dashboard:

# VERSION WITH TABLE (only partially working)

* Note: Uses servable(area='sidebar')
* Table not linked

In [ ]:
ls = hv.link_selections.instance()

# Table is not yet dynamically linked to the linked selection (still thinking about the best way to do this)
table = filtered_subrange[['time', 'place', 'mag', 'depth']].pipe(ls.filter).pipe(pn.widgets.Tabulator, 
                                                                         pagination='remote', page_size=10)

mag_hist = filtered_subrange.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200
)

depth_hist = filtered_subrange.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200
)

geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500
)


filtered_subrange.widgets().servable(area='sidebar')

pn.Column(
    hv.element.tiles.ESRI() * ls(geo.holoviews()),
    pn.Row(ls(depth_hist.holoviews()), ls(mag_hist.holoviews()), table.panel()), 
    sizing_mode='stretch_width'
).servable(title='Earthquake Interactive Demo')